try different samples of data for the train test split settings 

In [ ]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns

# metrics
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import classification_report, confusion_matrix

# transformations
from sklearn.feature_selection import RFE
from sklearn.preprocessing import MinMaxScaler

# models
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.multiclass import OneVsRestClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
score_dict = {}
def score_me(mn, mc, pn, y_test):
    """ 
    mn : the model name you want to have (ie. mn='Linear Regression')
    mc : the variable name for the model (ie. mc=linear_regression)
    pn : predictor variable name (ie. pn=y_pred)
    """
    global model_name
    model_name = {}
    model_name['Accuracy Score'] = round(accuracy_score(y_test, pn), 4)
    model_name['Micro F1 Score'] = round(f1_score(y_test, pn, average='micro'), 4)
    model_name['Macro F1 Score'] = round(f1_score(y_test, pn, average='macro'), 4)
    model_name['Weighted F1 Score'] = round(f1_score(y_test, pn, average='weighted'), 4)
    model_name['Micro Precision Score'] = round(precision_score(y_test, pn, average='micro'), 4)
    model_name['Macro Precision Score'] = round(precision_score(y_test, pn, average='macro'), 4)
    model_name['Weighted Precision Score'] = round(precision_score(y_test, pn, average='weighted'), 4)
    model_name['Micro Recall Score'] = round(recall_score(y_test, pn, average='micro'), 4)
    model_name['Macro Recall Score'] = round(recall_score(y_test, pn, average='macro'), 4)
    model_name['Weighted Recall Score'] = round(recall_score(y_test, pn, average='weighted'), 4)
    score_dict[mn] = model_name 
    print(classification_report(y_test, pn), '\n', '\n')
    
    plt.figure(figsize = (20, 8))
    sns.heatmap(pd.DataFrame(confusion_matrix(y_test, pn)), annot=True, fmt='g', annot_kws={"size": 15})
    plt.title(str(mn) +' Confusion Matrix')
    plt.ylabel('Actual Label')
    plt.xlabel('Predicted Labels')
    plt.tight_layout()

    plt.show()


In [ ]:

def Decision_Tree(X_train, y_train, X_test, y_test):
    global dctc
    dctc_start = time.time()
    dctc= DecisionTreeClassifier(random_state = 42).fit(X_train, y_train)
    dctc_predictions = dctc.predict(X_test)
    score_me('Decision Tree Classifier', dctc, dctc_predictions, y_test)

    dctc_end = time.time()
    compute_time_dctc = dctc_end - dctc_start
    model_name['Run Time(secs)'] = round(compute_time_dctc, 3)
    print('Scores:', score_dict['Decision Tree Classifier'])


In [ ]:
def RF_Feature_Importance(X_train, y_train, X_test, y_test):    
    rfc_feature_importances = pd.DataFrame(rfc.feature_importances_, index = X_train.columns,
                                         columns = ['Importance']).sort_values('Importance', ascending = False)
    plt.figure(figsize = (20, 8))
    sns.barplot(x = 'Importance', y = rfc_feature_importances.index,
            data = rfc_feature_importances)
    plt.title('Random Forest Classifier Feature Importances', fontsize = 14)
    plt.xlabel('Importance Value', fontsize = 12)
    plt.show()
    
    

def Feature_Optimization_RF(X_train, y_train, X_test, y_test ):
    results = pd.DataFrame(columns = ['Number of Features', 'Accuracy Score','Micro F1 Score','Macro F1 Score', 'Weighted F1 Score', 'Micro Precision Score', 'Macro Precision Score', 
                                     'Weighted Precision Score', 'Micro Recall Score', 'Macro Recall Score', 'Weighted Recall Score'])


    for index in np.arange(len(X_train.columns)):
        sel = RFE(RandomForestClassifier(random_state = 42, n_jobs = -1), n_features_to_select = index + 1)
        sel.fit(X_train, y_train)
        x_train_rfe = sel.transform(X_train)
        x_test_rfe = sel.transform(X_test)
        model = RandomForestClassifier(n_estimators = 50, random_state = 42, n_jobs = -1)
        model.fit(x_train_rfe, y_train)
        results.loc[index] = [index + 1,
                              round(accuracy_score(y_test, model.predict(x_test_rfe)), 4),
                              round(f1_score(y_test, model.predict(x_test_rfe), average='micro'), 4),
                              round(f1_score(y_test, model.predict(x_test_rfe), average='macro'), 4),
                              round(f1_score(y_test, model.predict(x_test_rfe), average='weighted'), 4),
                              round(precision_score(y_test, model.predict(x_test_rfe), average='micro'), 4),
                              round(precision_score(y_test, model.predict(x_test_rfe), average='macro'), 4),
                              round(precision_score(y_test, model.predict(x_test_rfe), average='weighted'), 4),
                              round(recall_score(y_test, model.predict(x_test_rfe), average='micro'), 4),
                              round(recall_score(y_test, model.predict(x_test_rfe), average='macro'), 4),
                              round(recall_score(y_test, model.predict(x_test_rfe), average='weighted'), 4)]
        return results 




In [ ]:

def Random_Forest(X_train, y_train, X_test, y_test):
    global rfc
    '''returns rf_results dataframe which must be saved '''
    rfc_start = time.time()
    rfc = RandomForestClassifier(n_estimators=50, random_state = 42, n_jobs = -1).fit(X_train, y_train)
    rfc_predictions = rfc.predict(X_test)
    score_me('Random Forest Classifier', rfc, rfc_predictions, y_test)

    rfc_end = time.time()
    compute_time_rfc = rfc_end - rfc_start
    model_name['Run Time(secs)'] = round(compute_time_rfc, 3)
    print('Scores:', score_dict['Random Forest Classifier'])
    

In [ ]:
def Gradient_Boosting_Classifier(X_train, y_train, X_test, y_test):
    global gbc
    gbc_start = time.time()


    gbc = GradientBoostingClassifier(n_estimators=10, random_state=42).fit(X_train, y_train)

    gbc_predictions = gbc.predict(X_test)
    score_me('Gradient Boosting Classifier', gbc, gbc_predictions, y_test)

    gbc_end = time.time()
    compute_time_gbc = gbc_end - gbc_start
    model_name['Run Time(secs)'] = round(compute_time_gbc, 3)
    print('Scores:', score_dict['Gradient Boosting Classifier'])


In [ ]:
def XG_Boost_Classifier(X_train, y_train, X_test, y_test):
    global xgc
    xgc_start = time.time()


    xgc = OneVsRestClassifier(XGBClassifier()).fit(X_train, y_train)

    xgc_predictions = xgc.predict(X_test)
    score_me('XG-Boost Classifier', xgc, xgc_predictions, y_test)

    xgc_end = time.time()
    compute_time_xgc = xgc_end - xgc_start
    model_name['Run Time(secs)'] = round(compute_time_xgc, 3)
    print('Scores:', score_dict['XG-Boost Classifier'])


In [ ]:
def KNN_Classifier(X_train, y_train, X_test, y_test):
    global knnc
    knn_start = time.time()
    
    knnc = KNeighborsClassifier(n_neighbors=7, n_jobs = -1).fit(X_train, y_train)
    knn_predictions = knnc.predict(X_test)
    score_me('K-Neighbors Classifier', knnc, knn_predictions, y_test)

    knn_end = time.time()
    compute_time_knn = knn_end - knn_start
    model_name['Run Time(secs)'] = round(compute_time_knn, 3)
    print('Scores:', score_dict['K-Neighbors Classifier'])


In [ ]:
def train_models(df, target_column, RF_O=False):
    X_train, X_test, y_train, y_test = train_test_split(df, target_column, random_state=42, test_size=0.25)
    
    Decision_Tree(X_train, y_train, X_test, y_test)
    
    Gradient_Boosting_Classifier(X_train, y_train, X_test, y_test)
    
    XG_Boost_Classifier(X_train, y_train, X_test, y_test)
    
    KNN_Classifier(X_train, y_train, X_test, y_test)
    Random_Forest(X_train, y_train, X_test, y_test)
    
    return pd.DataFrame.from_dict(score_dict)

    if RF_O == True:
    # shows a bar chart of feature importance
        RF_Feature_Importance(X_train, y_train, X_test, y_test )
    # returns a dataframe of how features affect the performance of the RF model 
        results = Feature_Optimization_RF(X_train, y_train, X_test, y_test )
        return results 
    

In [ ]:
score_dict_test_df = {}
def score_me_test_df(mn_t, mc_t, pn_t, y_test):
    """ 
    mn : the model name you want to have (ie. mn='Linear Regression')
    mc : the variable name for the model (ie. mc=linear_regression)
    pn : predictor variable name (ie. pn=y_pred)
    """
    global model_name_ts
    model_name = {}
    model_name['Accuracy Score'] = round(accuracy_score(y_test, pn_t), 4)
    model_name['Micro F1 Score'] = round(f1_score(y_test, pn_t, average='micro'), 4)
    model_name['Macro F1 Score'] = round(f1_score(y_test, pn_t, average='macro'), 4)
    model_name['Weighted F1 Score'] = round(f1_score(y_test, pn_t, average='weighted'), 4)
    model_name['Micro Precision Score'] = round(precision_score(y_test, pn_t, average='micro'), 4)
    model_name['Macro Precision Score'] = round(precision_score(y_test, pn_t, average='macro'), 4)
    model_name['Weighted Precision Score'] = round(precision_score(y_test, pn_t, average='weighted'), 4)
    model_name['Micro Recall Score'] = round(recall_score(y_test, pn_t, average='micro'), 4)
    model_name['Macro Recall Score'] = round(recall_score(y_test, pn_t, average='macro'), 4)
    model_name['Weighted Recall Score'] = round(recall_score(y_test, pn_t, average='weighted'), 4)
    score_dict_test_df[mn_t] = model_name 
    print(classification_report(y_test, pn_t), '\n', '\n')
    
    plt.figure(figsize = (20, 8))
    sns.heatmap(pd.DataFrame(confusion_matrix(y_test, pn_t)), annot=True, fmt='g', annot_kws={"size": 15})
    plt.title(str(mn_t) +' Confusion Matrix')
    plt.ylabel('Actual Label')
    plt.xlabel('Predicted Labels')
    plt.tight_layout()

    plt.show()


In [ ]:
def run_test_df(the_target_column, the_test_data):
    target_column = the_target_column
    set_to_test = the_test_data
    model_names = {'Decision Tree Classifier':[dctc, dctc.predict(set_to_test)],'Random Forest Classifier':[rfc, rfc.predict(set_to_test)],
                   'Gradient Boosting Classifier':[gbc, gbc.predict(set_to_test)],'XG-Boost Classifier':[xgc, xgc.predict(set_to_test)],
                   'K-Neighbors Classifier':[knnc, knnc.predict(set_to_test)]}
    for models in model_names:
        score_me_test_df(mn_t=models, mc_t=model_names[models][0], pn_t=model_names[models][1], y_test=the_target_column)

    model_results_ts = pd.DataFrame.from_dict(score_dict_test_df)
    return model_results_ts


In [ ]:
bd = pd.read_csv('balanced_dataframe_b.csv')
bd.info()
classifying_set = bd[['ARRIVAL_MONTH',
       'ARRIVAL_DAY', 'ARRIVAL_HOUR', 'ARRIVAL_MINUTE',
       'INCIDENT_MONTH',
       'INCIDENT_DAY', 'INCIDENT_HOUR', 'INCIDENT_MINUTE',
       'PROPERTY_USE_LABELS', 'ZIP_CODE_LABELS', 'INCIDENT_TYPE_DESC_LABELS']].astype('int16')




In [ ]:
train_models(classifying_set, target_column_fd, False)

In [ ]:
print(model_results)

In [ ]:

target_column_fd = bd['TARGET_CLASSES'].astype('int16')

test_set = pd.read_csv('test_model.csv')
test_set.info()
run_X_test = classifying_set
target_for_test = target_column_fd
fire_dept_test_results = run_test_df(target_for_test, run_X_test)


In [ ]:
fire_dept_test_results

In [ ]:
small = pd.read_csv('small_df_s.csv')
small_features = small.loc[:, small.columns != 'HIGHEST_LEVEL_DESC']
small_target = small.loc[:, 'HIGHEST_LEVEL_DESC']
small_results = train_models(small_features, small_target, False)
print(small_results)

In [ ]:
medium = pd.read_csv('medium_df_s.csv')
medium_features = medium.loc[:, medium.columns != 'HIGHEST_LEVEL_DESC']
medium_target = medium.loc[: 'HIGHEST_LEVEL_DESC']
medium_results = train_models(medium_features, medium_target, False)
print(medium_results)


In [ ]:
large = pd.read_csv('large_df_s.csv')
large_features = large.loc[:, large.columns != 'HIGHEST_LEVEL_DESC']
large_target = large.loc[: 'HIGHEST_LEVEL_DESC']
large_results = train_models(large_features, large_target, False)
print(large_results)
